In [124]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [125]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [126]:
test = pd.read_csv("/kaggle/input/titanic/test.csv")
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [127]:
from sklearn.ensemble import RandomForestClassifier



In [128]:
x=[]
for i in range(len(train['Cabin'])):
    y = ord(str(train['Cabin'][i])[0])
   
    if y==110:
        train['cabinas'] = 99999
    
    else:
        train['cabinas'] = y 
for i in range(len(test['Cabin'])):
    y = ord(str(test['Cabin'][i])[0])
   
    if y==110:
        test['cabinas'] = 99999
    
    else:
        test['cabinas'] = y        



In [129]:
train_test_data = [train, test] # combining train and test dataset
for dataset in train_test_data:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
features_drop = ['PassengerId','Name', 'Fare', 'Ticket','Cabin','Embarked']
test_copy = test
train = train.drop(features_drop, axis=1)
test = test.drop(features_drop, axis=1)
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)
X_train = train.drop('Survived', axis=1)
y_train = train['Survived']
train = train.drop('Survived',axis=1)
X_test = test
X = X_train
y = y_train
train.head() 

,Pclass,Sex,Age,SibSp,Parch,cabinas
0,3,0,22.0,1,0,99999
1,1,1,38.0,1,0,99999
2,3,1,26.0,0,0,99999
3,1,1,35.0,1,0,99999
4,3,0,35.0,0,0,99999


In [130]:
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=100,n_jobs=5)
clf.fit(X_train, y_train)
scores = cross_val_score(clf, X, y, cv=5)
print(scores.mean())
y_pred_random_forest = clf.predict(X_test)
acc_random_forest = round(clf.score(X_train, y_train) * 100, 2)
print (acc_random_forest)

0.8159876969430669
92.03


In [131]:
submission = pd.DataFrame({
        'PassengerId': test_copy["PassengerId"],
        'Survived': y_pred_random_forest
    })
submission.reset_index(drop=True, inplace=True) 
submission.to_csv('submission.csv',index=False)